In [ ]:
import brel
from brel.utils import pprint
from random import sample 
import pandas as pd

import brel_import
company = brel_import.Company("Mastercard", 1)
filing_row = company.df_filings.iloc[0]


In [ ]:
def search(filing_row):
    filing = filing_row["Filing"]
    components = filing.get_all_components()
    # print(components[0])
    # component_names = [component.get_URI().split("/")[-1] for component in components]
    component_names = [component.get_info() for component in components]

    # for i in component_names:
    #     print(i)

    from fuzzywuzzy import process

    # TODO: filter to concepts that I KNOW will be there (assets, net income, free cash flow)
    balancesheet = process.extract("BALANCE SHEETS", component_names, limit = len(component_names))
    incomestatement = process.extract("STATEMENTS OF COMPREHENSIVE INCOME", component_names,limit = len(component_names))
    cashflowstatement = process.extract("STATEMENTS OF CASH FLOWS", component_names,limit = len(component_names))

    print(balancesheet)    
    print(incomestatement)
    print(cashflowstatement)

    for component in components:
        if incomestatement[0][0] == component.get_info():
            return component
    
    # for i, x in enumerate(ratios):
    # # for ratio in ratios:
    #     print(component_names[i] + "   "+ str(x))

search(filing_row)

In [ ]:
# create two dfs, one with the header information for each file, like the filing date, possible components, possible dates 
# another with the actual data

def get_component_info_df(filing_row):
    # Get the cover page component.
    # cover_page_uri = "http://www.mycompany.com/roles/coverpage"
    filing = filing_row['Filing']
    print(filing.path)

    component = search(filing_row)

    if component.has_presentation_network():
        network = component.get_presentation_network()
        # pprint(network)

        # - put nodes (concepts) into table 
        master_table = []

        for node in network.get_all_nodes(): ### modified get_all_nodes function inplace so that it would maintain the same order as the ixbrl report
            report_element =node.get_report_element()
            concept = filing.get_concept(str(node.get_report_element()))

            if (concept):
                facts = filing.get_facts_by_concept(concept)
                labels = (concept.get_labels())
            else:
                facts = None
                labels = None


            # print(concept)
            # print(report_element
            #  + "    "+ concept)
            # print(facts)
            
            master_table.append({"report_element":report_element,"concept":concept,"facts":facts,"labels":labels})

        df = pd.DataFrame(master_table)
        display(df)
        return df

df_component = get_component_info_df(filing_row)



In [54]:
def afunc():
    df = df_component[["labels","facts"]]
    df["labels"] = df["labels"].map(lambda row: row[0] if not row==None else None)
    df["facts"] = df["facts"].map(lambda row: row[0].get_value() if not row==None else None)
    display(df)

    # print(df["facts"])

    # a = df['facts'].iloc[10].get_value()
    # print(a)

afunc()

C:\Users\matth\AppData\Local\Temp\ipykernel_3744\866997963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["labels"] = df["labels"].map(lambda row: row[0] if not row==None else None)
C:\Users\matth\AppData\Local\Temp\ipykernel_3744\866997963.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["facts"] = df["facts"].map(lambda row: row[0].get_value() if not row==None else None)


,labels,facts
0,None,None
1,Net Income,3011000000
2,None,None
3,Foreign currency translation adjustments,-168000000
4,Income tax effect,-9000000
5,"Foreign currency translation adjustments, net ...",-159000000
6,Translation adjustments on net investment hedges,47000000
7,Income tax effect,11000000
8,Translation adjustments on net investment hedg...,36000000
9,Cash flow hedges,22000000


In [ ]:
def afunc():
    for listoffacts in df_component['labels']:
        # print(fact)
        if listoffacts != None:
            # print(listoffacts)
            for fact in listoffacts:
                print(fact)
            print()

afunc()

print(filing_row['Filing'].path)